In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def full_connected(is_train):
    # 获取真实的数据
    mnist = input_data.read_data_sets("./data/mnist", one_hot=True)
    
    # 1. 建立数据占位符 x [None, 784] y_true [None, 10]
    with tf.variable_scope("data"):
        x = tf.placeholder(tf.float32, [None, 784])
        y_true = tf.placeholder(tf.int32, [None, 10])
    # 2. 建立一个全连接层网络 W [784, 10], b [10]
    with tf.variable_scope("fc_model"):
        # 随机初始化权重和偏置
        weight = tf.Variable(tf.random_normal([784, 10], mean=0.0, stddev=1.0), name="W")
        bias = tf.Variable(tf.constant(0.0, shape=[10]), name="bias")
        
        # 预测None个样本的输出结果matric，[None, 784]*[784,10]+[10] = [None,10]
        y_predict = tf.matmul(x, weight) + bias
        
    # 3. 求所有样本的损失，然后求平均值，均方误差损失
    with tf.variable_scope("softmax_loss"):
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=y_true, logits=y_predict, name="loss"))
        
    # 4. 梯度下降求出损失
    with tf.variable_scope("optimizer"):
        train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    # 5. 计算准确率
    with tf.variable_scope("accuracy"):
        equal_list = tf.equal(tf.arg_max(y_predict, 1), tf.arg_max(y_true, 1))
        
        # equal_list None个样本 [1, 0, 1, 0 ...]
        accuracy = tf.reduce_mean(tf.cast(equal_list, tf.float32))
    
    # 收集变量， 单个数值收集
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    
    # 高纬度变量收集
    tf.summary.histogram("weights", weight)
    tf.summary.histogram("bias", bias)
    
    # 定义一个初始化变量的op
    init_op = tf.global_variables_initializer()
    
    # 定义一个合并变量的op
    merged = tf.summary.merge_all()
    
    # 创建一个saver
    saver = tf.train.Saver()
    
    # 开启会话训练
    with tf.Session() as sess:
        # 初始化变量
        sess.run(init_op)
        
        # 建立events 文件，然后写入
        filewriter = tf.summary.FileWriter("/tf/logs/dnn_simple/", graph=sess.graph)
        
        print(x) 
        print(y_true) 
        if is_train == 1:
            for i in range(2000):
                # 取出真实存在的特征值和目标值
                mnist_x, mnist_y = mnist.train.next_batch(50)
                mnist_y = mnist_y.astype("int32")
                
                # 运行train_op 训练
                sess.run(train_op, feed_dict={x:mnist_x, y_true:mnist_y})
                
                # 写入每步训练的值
                summary = sess.run(merged, feed_dict={x:mnist_x, y_true:mnist_y})
                
                filewriter.add_summary(summary,i)
                
                print("训练第%d步 accuracy:%f" % (i, sess.run(accuracy, feed_dict={x:mnist_x, y_true:mnist_y})))
                
            # 保持模型
            ww =sess.run(weight, feed_dict={x:mnist_x, y_true:mnist_y})
            bb = sess.run(bias, feed_dict={x:mnist_x, y_true:mnist_y})
            saver.save(sess, "./ckpt/dnn_simple/fc_model")
        else:
            # 加载模型
            saver.restore(sess, "./ckpt/dnn_simple/fc_model")
            
            # 预测 10 张图片
            for i in range(10):
                # 每次测试一张图片 x_test:[1, 784]  y_test:[1,10]
                # type(x): numpy.ndarray
                x_test, y_test = mnist.test.next_batch(1)
                print("第%d张图片，y_true:%d, y_predict:%d" % (
                      i,
                      tf.argmax(y_test, 1).eval(),
                      tf.argmax(sess.run(y_predict, feed_dict={x:x_test, y_true:y_test}), 1).eval()
                      ))
             
            x_test, y_test = mnist.test.next_batch(100)
            y_100 = sess.run(y_predict, feed_dict={x:x_test,y_true:y_test})
            print("100:", tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_test, 1), tf.argmax(y_100, 1)), tf.float32)).eval())
            ww =sess.run(weight, feed_dict={x:x_test, y_true:y_test})
            bb = sess.run(bias, feed_dict={x:x_test, y_true:y_test})
    return ww, bb
        

In [ ]:
ww, bb = full_connected(1)

In [ ]:
bb

In [4]:
ww, bb = full_connected(0)
bb

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Tensor("data_1/Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("data_1/Placeholder_1:0", shape=(?, 10), dtype=int32)


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key fc_model_1/W not found in checkpoint
	 [[node save_1/RestoreV2 (defined at <ipython-input-2-572967cccab9>:50) ]]

Original stack trace for 'save_1/RestoreV2':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-p

In [ ]:
# 标准正态分布 mean=0， std=1
print(ww.mean())
print(ww.std())

In [ ]:
a = tf.constant(1.0, shape=[3,2])
print(a)

with tf.Session() as sess:
    print(sess.run(a))


In [6]:
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [18]:
mnist.train.images[0].dtype
mnist.train.labels[0]
# mnist.train.labels.shape
x, y = mnist.test.next_batch(1)
print(type(x))
print(y)
with tf.Session() as sess:
    print(tf.argmax(y,1).eval())


<class 'numpy.ndarray'>
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[0]


In [ ]:
mnist.test.images.shape
a = mnist.train.next_batch(10)[1]
a.astype("int32")
a.shape

In [ ]:
indices = [0,0, 1, 1, 2, 3]
depth = 4
a = tf.one_hot(indices, depth)
with tf.Session() as sess:
    print(sess.run(a))
    

In [5]:
import numpy as np

# a = tf.random_normal(shape=[3,6],dtype=tf.float16)
aa = np.random.randint(0, 30, (3,6)) 
print(aa)
a = tf.constant(aa)
print(tf.argmax(a))
with tf.Session() as sess:
#     print(a.eval())
    print(tf.argmax(a, 1).eval())

[[ 9 17 10 24 12 22]
 [10 26 10  6  6 12]
 [ 9 16  9 28 13 11]]
Tensor("ArgMax_3:0", shape=(6,), dtype=int64)
[3 1 3]


In [ ]:
a = tf.constant([1,2,3,4, 6, 7])
b = tf.constant([1,2,0,4, 5, 7])
equal_list = tf.equal(a,b)
equal_percent = tf.reduce_mean(tf.cast(equal_list, tf.float32))

with tf.Session() as sess:
    print(equal_list.eval())
    print(equal_percent.eval())